In [8]:
import pandas as pd

# Membaca dataset
dataset = pd.read_csv('C:/Users/arsen/Healthylicious/data/cleaned/csv/recipes_cleaned.csv')
# columns_to_drop = ['ID']
# dataset = dataset.drop(columns=columns_to_drop)



In [9]:
# Mengecek nilai None (missing values)
missing_values = dataset.isnull().sum()
print("Nilai None (missing values) per kolom:")
print(missing_values)

Nilai None (missing values) per kolom:
Category             0
Title                0
Total Time           0
All Ingredients      0
Ingredient Groups    0
Instructions         0
Nutrition            0
Cuisine              0
Yields               0
Image                0
Ratings              0
Description          0
Status               0
dtype: int64


In [10]:
# Menghapus baris yang mengandung nilai None
dataset_cleaned = dataset.dropna()

In [11]:
duplicate_rows = dataset_cleaned[dataset_cleaned.duplicated(subset='Title', keep='first')]

# Menghitung jumlah duplikat
num_duplicates = duplicate_rows.shape[0]

print(f"Jumlah baris yang memiliki title duplikat: {num_duplicates}")

Jumlah baris yang memiliki title duplikat: 0


In [12]:
df_cleaned = dataset_cleaned.drop_duplicates(subset='Title', keep='first')

# Menyimpan dataset yang telah dibersihkan
df_cleaned.to_csv('recipes_cleaned.csv', index=False)

In [13]:
# Mengecek nilai duplikat
duplicates = dataset_cleaned.duplicated().sum()
print(f"Jumlah baris duplikat: {duplicates}")

Jumlah baris duplikat: 0


In [15]:
# Menghapus baris duplikat
dataset_cleaned = dataset_cleaned.drop_duplicates()

In [16]:
dataset_cleaned['recipeId'] = range(1, len(dataset_cleaned) + 1)
dataset_cleaned = dataset_cleaned[['recipeId'] + [col for col in dataset_cleaned.columns if col != 'ID']]

In [18]:
dataset_cleaned.head()

,recipeId,Category,Title,Total Time,All Ingredients,Ingredient Groups,Instructions,Nutrition,Cuisine,Yields,Image,Ratings,Description,Status,recipeId
0,1,Main Course,Swedish Meatballs Recipe,30,"1 pound ground beef, 2 tablespoons diced onion...",IngredientGroup(ingredients=['1 pound ground b...,For the meatballs:\r\nMix together all ingredi...,"{'calories': '220 kcal', 'carbohydrateContent'...",American,6 servings,https://addapinch.com/wp-content/uploads/2022/...,4.95,The Best Swedish Meatballs feature tender meat...,Complete,1
1,2,"Appetizer,Main Course",Baked Crispy Buffalo Chicken Wings Recipe,60,"6 pounds chicken wings (trimmed), 3 tablespoon...",IngredientGroup(ingredients=['6 pounds chicken...,Preheat the oven to 425º F. Line a baking shee...,"{'servingSize': '0.25 pound', 'calories': '419...",American,8 servings,https://addapinch.com/wp-content/uploads/2018/...,5.00,Crispy Baked Buffalo Chicken Wings that are so...,Complete,2
2,3,Appetizer,Guacamole Recipe,5,"2 avocados (halved, seeded, and peeled), 1 tab...",IngredientGroup(ingredients=['2 avocados (halv...,Mash avocado with a fork or potato masher in a...,"{'carbohydrateContent': '7 g', 'proteinContent...",Mexican,6 servings,https://addapinch.com/wp-content/uploads/2013/...,4.50,"This easy Guacamole recipe, made with fresh in...",Complete,3
3,4,Appetizer,Perfect Queso Recipe,5,"1/2 cup pepper jack cheese, 1 cup grated white...",IngredientGroup(ingredients=['1/2 cup pepper j...,Stovetop Queso:\r\nAdd cheeses and half of the...,"{'calories': '534 kcal', 'carbohydrateContent'...",American,2 servings,https://addapinch.com/wp-content/uploads/2015/...,5.00,This easy and delicious Queso recipe is made w...,Complete,4
4,5,Appetizer,Buffalo Chicken Dip Recipe,25,"2 cups chicken breasts (cooked and shredded), ...",IngredientGroup(ingredients=['2 cups chicken b...,Preheat oven to 350º F.\r\nMix together all in...,"{'calories': '106 kcal', 'proteinContent': '4 ...",American,12 servings,https://addapinch.com/wp-content/uploads/2024/...,5.00,Buffalo Chicken Dip Recipe is the best easy ap...,Complete,5


In [19]:
dataset_cleaned.to_csv('combined_ingredients.csv', index=False)

print("Dataset telah dibersihkan dari nilai None dan duplikat.")

Dataset telah dibersihkan dari nilai None dan duplikat.


### Cleaning Ingredients


In [20]:
import pandas as pd
import numpy as np
import string, re


import nltk
nltk.download('stopwords')

import spacy
nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arsen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
df = pd.read_csv("C:/Users/arsen/Healthylicious/data/cleaned/csv/recipes_cleaned.csv")

In [22]:
df = df.dropna(subset=['Title'])  

In [23]:
df['ID'] = range(1, len(df) + 1)
df = df.set_index('ID')

In [24]:
measure_words = ['bottle', 'bottles', 'box', 'boxes', 'bunch','bunches', 'bushel','bushels', 
                'can', 'cans', 'container', 'c', 'cup', 'cups', 'carton', 'cartons', 'half',
                'dash','dashes', 'drop','drops','fl', 'fl.', 'fluid','jar', 'jars','ounce','ounces','oz',
                'g', 'gallon','gallons', 'glass','glasses', 'gram','grams','kg','kgs', 'lb','lbs', 'inch', 'inches'
                'liter','liters', 'l', 'large', 'medium', 'ml','mls', 'package','pkg','small', 'to taste',
                'pinch','pinches', 'pint','pints', 'pound','pounds', 'qt','qts', 'quart','quarts',
                'scoop','scoops', 'sliced','slivered','stick','sticks', 'tablespoon','tablespoons',
                'tbs','tbsp','tbsps', 'teaspoon','teaspoons','tsp','tsps','whole','cut into cube','cut into piece','fully'
                ,'optional mix in','optional topping', 'package', 'piece','bags','ball','cut']

In [25]:
def clean_once(text):
    # Remove anything inside parentheses, including the parentheses themselves
    text = re.sub(r'\([^)]*\)', '', text)
    
    text = re.sub(r"confectioner'?s'", 'confectioners', text)
    text = re.sub(r"confectioner'?s", 'confectioners', text)
    text = re.sub(r"hershey'?s", 'hersheys', text)
    text = re.sub(r"hershey'?s'", 'hersheys', text)
    
    # Replace "half-and-half" with "halfandhalf"
    text = text.replace('half-and-half', '')
    text = text.replace('peanut butter chocolate chip cookie', 'peanut butter, chocolate chip, cookie')
    text = text.replace('the traditional decoration is ', '')
    text = text.replace('of sauce', '')
    text = text.replace('portabella', 'portobello')
    text = text.replace('portabello', 'portobello')
    text = text.replace('pears', 'pear')
    text = text.replace('halfandhalf', '')
    text = text.replace('at room temperature', '')
    text = text.replace('for the topping', '')
    text = text.replace('broken into piece', '')
    text = text.replace('breadcrumb', 'bread crumb')
    text = text.replace('spring mix salad green', 'spring salad green')
    text = text.replace('spring salad green mix', 'spring salad green')
    text = text.replace('tomatoes', 'tomato')
    text = text.replace('crabmeat', 'crab meat')
    text = text.replace('bleu', 'blue')
    text = text.replace('gruyère', 'gruyere')
    text = text.replace('bbq', 'barbeque')
    text = text.replace('store bought', '')
    text = text.replace('another bread such as sourdough' , 'sourdough')
    text = text.replace('frozen ' , '')

    
    # Remove fraction characters
    text = re.sub(r'[\u00BC-\u00BE\u2150-\u215E]', '', text)
    
    ingredlist = []
    for ingred in text.split(", "):
        # Remove measurements like "8-ounce"
        ingred = re.sub(r'\d+-ounce ', '', ingred)
        # Remove digits
        ingred = re.sub(r'\d', '', ingred)
        ingred = re.sub('\w*\d\w*', ' ', ingred)
        ingred = re.sub(r'\s+', ' ', ingred)  # Remove extra whitespace
        ingred = re.sub('[%s]' % re.escape(string.punctuation), ' ', ingred)
        ingred = ingred.replace('/', ' ')     # Remove slashes
        ingred = ingred.replace(')', '')      # Remove closing parenthesis
        ingred = ingred.replace('.', '')      # Remove dot
        ingred = ingred.replace('-', '')      # Remove hyphen
        ingred = ingred.replace('"', '')      # Remove hyphen
        ingred = ingred.replace("'", '')      # Remove hyphen
        ingred = ingred.replace("&", '')      # Remove hyphen
        ingred = ingred.replace(' and ', ', ')     # Replace " and " with ", "
        ingred = ingred.replace('flour for', ', ')     # Replace " and " with ", "
        ingred = ingred.replace('of sauce', '')     # Replace " and " with ", "
        ingred = ingred.replace(' stuffed with ', ', ')     # Replace " and " with ", "
        ingred = ingred.replace('another bread such as ' , '')     # Replace " and " with ", "
        ingred = ingred.replace(' about ', ', ')     # Replace " and " with ", "
        ingred = ingred.replace(' with ', ', ')     # Replace " and " with ", "
        ingred = ingred.replace(' roll ', ', ')     # Replace " and " with ", "
        ingred = ingred.replace(' or ', ', ')     # Replace " and " with ", "
        ingred = ingred.replace(' in ', ', ')     # Replace " and " with ", "
        ingred = ' ' + ingred + ' '# Padding in case measure_word is first or last
        for measure_word in measure_words:    # Remove measure words
            ingred = ingred.replace(' ' + measure_word + ' ', ' ')
        ingred = ingred.replace(' or ', ', ') # Replace " or " with ", "
        ingred = ingred.strip().lower()       # Strip leading/trailing whitespace and convert to lowercase
        ingredlist.append(ingred)
        cleaned_text = ', '.join(ingredlist)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Ensure single spaces
        cleaned_text = re.sub(r'\b(\w+)\s*,\s*\1\b', r'\1', cleaned_text)  # Remove repeated words separated by commas
        cleaned_text = re.sub(r',\s*,', ',', cleaned_text)  # Remove extra commas
        cleaned_text = re.sub(r'\s*,\s*', ', ', cleaned_text.strip())  # Ensure proper comma spacing
    return cleaned_text

In [26]:
df['Ingredients'] = df['All Ingredients'].apply(clean_once)

In [27]:
def word_singularizer(word):
    nlp_word = nlp(word)[0]
    lemma = nlp_word.text
    if nlp_word.tag_ in {"NNS", "NNPS"}:
        lemma = nlp_word.lemma_
    return lemma

def text_singularizer(text):
    ingredlist = []
    for ingred in text.split(', '):
        new_list = []
        for word in ingred.split():
            new_list.append(word_singularizer(word))
        ingred = ' '.join(new_list)
        ingredlist.append(ingred)        
    return ', '.join(ingredlist)

In [28]:
df['Ingredients'] = df['Ingredients'].fillna('')

In [29]:
unwanted_phrases = ['red skinned','flour for','for the topping','royal blue gel paste ','quick cooking','quick cook','cut into cube','for griddle','cream of chicken soup cream of chicken soup of','cream of','condensed cream of','to drizzle','cut into','fat free ','on the cob','dripping from','easy blender','dressing of choice', 'each tortellini','store bought soup','equal size', 'm ms','for topping', 'for rolling cookie','broken into chunks','bulbs of', 'with juice', 'more for tops','box duncan hines','bone in','box duncan hine',
                    'egg shaped','about that','add in','gluten free','day glazed', 'package unflavored', 'candy coated','strawberry flavored','stuffed with','slices thin', 'thinly sliced', 'all purpose','boneless skinless', 'half-and-half', 'thick-cut', 'thick cut', 'finely grated','hard boiled', 'pieces cooked', 'for garnish','to garnish', 'to cover', 'zest of','the traditional decoration is','frozen','into cube','into', 'thawed',
                    'restaurant style','store bought','other decoration','black bean black bean','optional mix in','optional topping', 'the best','round of','long grain', 'low sugar', 'low sodium','head of', 'a little for handling','another bread such as','oven cooked','cooked shredded smoked','hard cooked', 'grated fresh','strong brewed', 'for deep fried', 'if used', 'in water', 'equal size','a little for handling', 'for deep frying', 'of espresso powder','of sauce','of sugar', 'top round']
unwanted_words = ['cooked','','coarsely','white','yellow','deviled','full','freshly','head','heaping','petite','pure','roast','cold','sliced','soft','spring','spiral','warm','heavy','cut','store bought','tablespoon','storebought','about', 'favourite', 'favorite','breading','cream of chicken soup of water','bulb','jellied','reserved','herbed','sized','unsulphured','marinated','rubbed','grilled','stewed','mixed','undrained', 'colored','shelled','toasted', 'chilled','glazed','coated','malted', 'xl sized', 'unbaked','unflavored','creamed', 'sauuted', 'overripe', 'ripe', 'spread','rolled', 'granulated', 'hot', 'packed', 'plain', 'thinly', 'dried', 'unsweetened', 'roasted', 'boneless', 'skinless', 'cubed', 'minced', 'crumbled', 'assorted', 'softened', 'sharp',  'boiling', 'rinsed', 'drained', 'crushed', 'popped', 'thin', 'light', 'raw', 'natural', 'oven-cooked', 
                 'deseeded','grapeseed', 'cooked', 'steamed','roughly','black eyed','shredded', 'smoked', 'homemade', 'reduced', 'mild',  'baked', 'seedless', 'new', 'old', 'melted', 'pieces', 'finely', 'twirl', 'boiled', 'lump', 'pulled', 'blackened', 'bag', 'sundried', 'prepared', 'whipped', 'dry', 'wet', 'uncooked', 'recipe','package','packet','pan','regular','pan','roll','s','loaf',
                 'frozen','pickled', 'mix','frosting','grated','grilled', 'packed', 'large', 'small', 'medium', 'fresh', 'ground', 'diced', 'chopped', 'neutral', 'slider', 'sheet','boneless', 'breasts', 'breast', 'skinless', 'salted', 'filled', 'brewed','dried','shaved','refried', 'canned', 'mashed', 'evaporated', 'seasoned', 'refigerated', 'distilled', 'caramelized','to', 'xl','bags','ball', 'cup','jumbo','mini','of sauce','one'
                 'thawed','crispy', 'firmly','fine','english','yolk','active','violet gel paste ','the traditional decoration is','sweet','round','ever','peeled', 'pillsbury','cap','cutlet','from', 'flaked',  'pasteurized','pittled', 'sweetened', 'seasoned','half','another bread such as', 'best', 'unsalted','thick','thin','with bone in', 'boil','unpilled', 'fried', 'halved', 'canned', 'refrigerated', 'one','refried', 'preferred', 'slice', 'fashioned', 'quarted', 'quartered', 'sweetened', 'unpeeled', 'pitted', 'deveined','perfect','piece','pinch']


def remove_unwanted_words_and_phrases(text):
    # Remove unwanted phrases
    for phrase in unwanted_phrases:
        text = re.sub(r'\b' + re.escape(phrase) + r'\b', '', text)
    
    # Remove unwanted words
    words = text.split(', ')
    cleaned_words = []
    for word in words:
        tokens = word.split()
        cleaned_tokens = [token for token in tokens if token not in unwanted_words]
        cleaned_words.append(' '.join(cleaned_tokens))
    
    # Join cleaned words and handle any extra spaces or commas
    cleaned_text = ', '.join(cleaned_words).replace(' ,', ',').strip()
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text = re.sub(r',\s*,', ',', cleaned_text)
    cleaned_text = re.sub(r'\s*,\s*', ', ', cleaned_text)
    
    return cleaned_text

In [30]:
teststr = df['Ingredients'][172]

teststr

'cream cheese, butter, confectioners sugar, molasses, vanilla extract, ginger, ground cinnamon, toffee bits'

In [31]:
print("Preprocessed:\n", teststr, '\n')
singularized_text = text_singularizer(teststr)
print("Singularized:\n", singularized_text, '\n')
unwanted_words_removed_text = remove_unwanted_words_and_phrases(singularized_text)
print("Unwanted Words and Phrases Removed:\n", unwanted_words_removed_text, '\n')

Preprocessed:
 cream cheese, butter, confectioners sugar, molasses, vanilla extract, ginger, ground cinnamon, toffee bits 

Singularized:
 cream cheese, butter, confectioner sugar, molasses, vanilla extract, ginger, ground cinnamon, toffee bit 

Unwanted Words and Phrases Removed:
 cream cheese, butter, confectioner sugar, molasses, vanilla extract, ginger, cinnamon, toffee bit 



In [32]:
df['Ingredients'] = df['Ingredients'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [33]:
  # Takes 2 min per 1000 rows
df['IngredientsProcessed'] = df['Ingredients'].apply(text_singularizer)# Takes 1 min per 1000 rows
df['IngredientsRemovedAdj'] = df['IngredientsProcessed'].apply(remove_unwanted_words_and_phrases)# Takes 1 min per 1000 rows
df.head()

,Category,Title,Total Time,All Ingredients,Ingredient Groups,Instructions,Nutrition,Cuisine,Yields,Image,Ratings,Description,Status,Ingredients,IngredientsProcessed,IngredientsRemovedAdj
ID,,,,,,,,,,,,,,,,
1,Main Course,Swedish Meatballs Recipe,30,"1 pound ground beef, 2 tablespoons diced onion...",IngredientGroup(ingredients=['1 pound ground b...,For the meatballs:\r\nMix together all ingredi...,"{'calories': '220 kcal', 'carbohydrateContent'...",American,6 servings,https://addapinch.com/wp-content/uploads/2022/...,4.95,The Best Swedish Meatballs feature tender meat...,Complete,"ground beef, diced onion, egg, stone house sea...","ground beef, diced onion, egg, stone house sea...","beef, onion, egg, stone house seasoning, allsp..."
2,"Appetizer,Main Course",Baked Crispy Buffalo Chicken Wings Recipe,60,"6 pounds chicken wings (trimmed), 3 tablespoon...",IngredientGroup(ingredients=['6 pounds chicken...,Preheat the oven to 425º F. Line a baking shee...,"{'servingSize': '0.25 pound', 'calories': '419...",American,8 servings,https://addapinch.com/wp-content/uploads/2018/...,5.00,Crispy Baked Buffalo Chicken Wings that are so...,Complete,"chicken wings, cornstarch, stone house seasoni...","chicken wing, cornstarch, stone house seasonin...","chicken wing, cornstarch, stone house seasonin..."
3,Appetizer,Guacamole Recipe,5,"2 avocados (halved, seeded, and peeled), 1 tab...",IngredientGroup(ingredients=['2 avocados (halv...,Mash avocado with a fork or potato masher in a...,"{'carbohydrateContent': '7 g', 'proteinContent...",Mexican,6 servings,https://addapinch.com/wp-content/uploads/2013/...,4.50,"This easy Guacamole recipe, made with fresh in...",Complete,"avocados, lime juice, sea salt, clove garlic, ...","avocados, lime juice, sea salt, clove garlic, ...","avocados, lime juice, sea salt, clove garlic, ..."
4,Appetizer,Perfect Queso Recipe,5,"1/2 cup pepper jack cheese, 1 cup grated white...",IngredientGroup(ingredients=['1/2 cup pepper j...,Stovetop Queso:\r\nAdd cheeses and half of the...,"{'calories': '534 kcal', 'carbohydrateContent'...",American,2 servings,https://addapinch.com/wp-content/uploads/2015/...,5.00,This easy and delicious Queso recipe is made w...,Complete,"pepper jack cheese, grated white american chee...","pepper jack cheese, grated white american chee...","pepper jack cheese, american cheese, queso che..."
5,Appetizer,Buffalo Chicken Dip Recipe,25,"2 cups chicken breasts (cooked and shredded), ...",IngredientGroup(ingredients=['2 cups chicken b...,Preheat oven to 350º F.\r\nMix together all in...,"{'calories': '106 kcal', 'proteinContent': '4 ...",American,12 servings,https://addapinch.com/wp-content/uploads/2024/...,5.00,Buffalo Chicken Dip Recipe is the best easy ap...,Complete,"chicken breasts, cream cheese, ranch dressing,...","chicken breast, cream cheese, ranch dressing, ...","chicken, cream cheese, ranch dressing, buffalo..."


In [34]:
def remove_duplicates(recipe):
    ingredients = recipe.split(', ')
    unique_ingredients = list(dict.fromkeys(ingredients))  # Remove duplicates while maintaining order
    return ', '.join(unique_ingredients)

# Apply the function to each row in the 'Ingredients' column
df['IngredientsRemovedAdj'] = df['IngredientsRemovedAdj'].apply(remove_duplicates)

In [35]:
ingredient_mapping = {
    'chocolate chip chocolate bar':'chocolate',
    'air fryer potato':'pototo',
    'almond extract':'almond',
    'another bread such as sourdough':'sourdough',
    'apple cider concentrate':'apple cider',
    'applesauce':'apple sauce',
    'artichoke heart':'artichoke',
    'asparagus spear':'asparagus',
    'avocado puree':'avocado',
    'avocados':'avocado',
    'bacon dripping':'bacon',
    'bacon jam':'bacon',
    'bacon vinaigrette':'bacon',
    'bake shortbread cookie':'shortbread cookie',
    'basil leaves':'basil',
    'balsamic roast beef':'balsamic',
    'beef stock':'beef broth',
    'blue cheese crumble':'blue cheese',
    'breakfast sausage':'sausage',
    'broccoli floret':'broccoli',
    'broccoli slaw':'broccoli',
    'pork chop': 'pork',
    'pork butt': 'pork',
    'butter lettuce leaves': 'butter lettuce',
    'canola oil': 'canola',
    'caramel sauce': 'caramel',
    'carrot carrot': 'carrot',
    'celery leaves': 'celery',
    'cherry pie filling': 'cherry pie',
    'chewy chocolate chip cookie': 'cookie',
    'chewy ginger molasses cookie': 'cookie',
    'chicken leg': 'chicken',
    'chicken tender': 'chicken',
    'chicken thigh': 'chicken',
    'chicken wing': 'chicken',
    'chicken wing': 'chicken',
    'chocolate bar': 'chocolate',
    'chocolate chip': 'chocolate',
    'chocolate chunks': 'chocolate',
    'chocolate ganache filling': 'chocolate ganache',
    'chocolate graham cracker crumb': 'chocolate ganache cracer',
    'chocolate ice cream': 'ice cream',
    'chocolate kiss candy': 'chocolate candy',
    'chocolate sugar cookie': 'chocolate cookie',
    'chocolate wafer cookie': 'chocolate wafer',
    'chuck roast': 'chuck',
    'clove garlic': 'garlic',
    'cloves garlic': 'garlic',
    'coca cola bbq sauce': 'coca cola',
    'cocktail peanut': 'peanut',
    'cocktail sausage': 'sausage',
    'cocoa mix': 'cocoa',
    'cocoa powder': 'cocoa',
    'coconut amino': 'coconut',
    'coconut flake': 'coconut',
    'coleslaw mix': 'coleslaw',
    'collard green leaves': 'collard green',
    'cookie icing': 'cookie',
    'cookie dough dip': 'cookie',
    'corn kernel niblet corn': 'corn kernel',
    'cornstarch': 'corn starch',
    'zucchini squash': 'zucchini',
    'worcestershire sauce or coconut amino': 'worcestershire sauce',
    'wholemeal flour': 'flour',
    'white queso cheese': 'quesso cheese',
    'white chocolate chip morsel': 'chocolate chip',
    'white chocolate candy melt melting wafer': 'white chocolate candy',
    'white chocolate buttercream': 'chocolate buttercream',
    'white cheddar cheese': 'cheddar cheese',
    'white bean': 'bean',
    'white balsamic vinegar': 'balsamic vinegar',
    'whipping cream cheese': 'whipping cream',
    'violet gel paste food color': 'food color',
    'vinaigrette dressing': 'vinaigrette',
    'vegetable vegetable': 'vegetable',
    'vanilla bean paste': 'vanilla bean',
    'venison tenderloin': 'venison',
    'venison roast': 'venison',
    'turnip green': 'turnip',
    'tortilla chip': 'tortilla',
    'tomato tomato': 'tomato',
    'tomato vinaigrette': 'vinaigrette',
    'tomatillos': 'tomatillo',
    'tabasco': 'tabasco',
    'sugar sprinkles': 'sugar',
    'strip bacon dripping': 'bacon',
    'strip bacon': 'bacon',
    'sugar cookie dough': 'sugar cookie',
    'streusel topping': 'streusel',
    'strawberry jell o': 'strawberry jello',
    'sriracha sauce': 'sriracha',
    'sriracha sauce': 'sriracha',
    'sriracha chili sauce': 'sriracha',
    'sprinkle fine sea salt': 'sea salt',
    'spinach spinach': 'spinach',
    'spaghetti noodle' : 'spaghetti',
    'spaghetti squash' : 'spaghetti',
    'southern caramel icing' : 'caramel',
    'skirt' : 'skirt steak',
    'sirloin steak' : 'sirloin',
    'shortbread pie crust' : 'shortbread crust',
    'sea salt flake' : 'sea salt',
    'salmon filet' : 'salmon fillet',
    'russet potato' : 'potato',
    'rosemary leaves' : 'rosemary',
    'roma potato' : 'potato',
    'red pepper flake' : 'red pepper',
    'red food coloring' : 'food coloring',
    'raspberry preserves' : 'raspberry',
    'raspberry preserves' : 'raspberry',
    'pumpkin puree pumpkin puree' : 'pumpkin puree',
    'pumpkin pie spice' : 'pumpkin pie',
    'portobellos' : 'portobello',
    'portobello mushroom' : 'portobello',
    'pork tenderloin' : 'pork',
    'pork sirloin' : 'pork',
    'pork shoulder' : 'pork',
    'pork loin' : 'pork',
    'pork breakfast sausage' : 'sausage',
    'pomegranate seed' : 'pomegranate',
    'pomegranate juice' : 'pomegranate',
    'plum tomato' : 'tomato',
    'pinto bean pinto bean' : 'pinto bean',
    'pineapple pineapple chunks' : 'pineapple',
    'pineapple pineapple' : 'pineapple',
    'pineapple jalapeno cream cheese' : 'pineapple',
    'pineapple chunks' : 'pineapple',
    'pimento pepper' : 'pimento',
    'pimento cheese' : 'pimento',
    'peppermint extract' : 'peppermint',
    'peppermint chocolate chip' : 'chocolate chip',
    'peppermint bark' : 'peppermint',
    'peppermint candy' : 'candy',
    'peanut butter chip' : 'peanut butter',
    'peach barbeque sauce' : 'barbeque sauce',
    'peach bbq sauce' : 'barbeque sauce',
    'parsley leave' : 'parsley',
    'parmesan' : 'parmesan cheese',
    'oreos' : 'oreo cookie',
    'oregano leaves' : 'oregano',
    'oregano flake' : 'oregano',
    'olive oil spray' : 'olive oil',
    'of sauce' : 'sauce',
    'nonstick cooking spray' : 'cooking spray',
    'naan' : 'naan bread',
    'mozzarella' : 'mozzarella cheese',
    'milk chocolate chip' : 'chocolate chip',
    'marshmallow creme' : 'marshmallow cream',
    'lima bean lima bean' : 'lima bean',
    'lettuce leaves' : 'lettuce',
    'lettuce green' : 'lettuce',
    'lemon curd' : 'lemon',
    'lemon juice' : 'lemon',
    'lemon lime soda' : 'lemon',
    'lemon sugar' : 'lemon',
    'lemon zest' : 'lemon',
    'korean chili pepper powder' : 'pepper',
    'jalapenos' : 'jalapeno',
    'jalapeño' : 'pepper',
    'hamburger bun' : 'bun',
    'ham bone' : 'ham hock',
    'gruyère cheese' : 'gruyere cheese',
    'green tomatoes' : 'tomato',
    'green tomato' : 'tomato',
    'gravy balsamic roast beef' : 'beef',
    'graham cracker crust' : 'graham cracker',
    'graham cracker crumb' : 'graham cracker',
    'grape tomato' : 'tomato',
    'gorgonzola crumble':'gorgonzola cheese',
    'gorgonzola cheese crumble':'gorgonzola cheese',
    'garlic clove':'garlic',
    'garlic cloves':'garlic',
    'food color':'food coloring',
    'flank steak':'flank',
    'espresso powder':'espresso',
    'egg white':'egg',
    'each tortellini tortellini':'tortellini',
    'cracker crumb':'cracker',
    'dijon mustard powder':'dijon mustard',
    'dark chocolate bar':'dark chocolate',
    'dark chocolate chunks':'dark chocolate',
    'crab meat':'crab',
    'country ham':'ham',
    'beef bouillon' : 'beef',
    'beef brisket' : 'beef',
    'beef tenderloin' : 'beef',
    'blue cheese dipping sauce' : 'blue cheese',
    'blue cheese dressing' : 'blue cheese',
    'candy cane':'candy',
    'candy corn':'candy',
    'candy melt':'candy',
    'candy pumpkins':'candy',
    'caramel candy':'candy',
    'catfish filet':'catfist',
    'chicken tenderloin':'chicken',
    'chocolate candy':'candy',
    'chocolate ganache cracer':'chocolate ganache',
    'coarse sugar':'sugar',
    'cranberry cranberry':'cranberry',
    'red pepper cream sauce':'cream sauce',
    'pretzel twist':'pretzel',
    'pototo':'potato',
    'mint chocolate chip ice cream':'ice cream',
    'lime juice':'lime',
    'lime wedge':'lime',
    'lime zest':'lime',
    'fettuccine':'fettucine',
    'chocolate candy melt melting wafer':'chocolate'
    
    # Add more mappings as needed
    # Add more mappings as needed
}

In [36]:
def generalize_ingredients(ingredients, mapping):
    # Split the ingredients by comma and strip any extra whitespace
    ingredients_list = [ingredient.strip() for ingredient in ingredients.split(',')]
    # Replace specific ingredients with their base forms
    generalized_ingredients = [mapping.get(ingredient, ingredient) for ingredient in ingredients_list]
    # Join the ingredients back into a string
    return ', '.join(generalized_ingredients)

In [37]:
df['IngredientsRemovedAdj'] = df['IngredientsRemovedAdj'].apply(lambda x: generalize_ingredients(x, ingredient_mapping))

In [38]:
import pandas as pd

# Assume df is your existing DataFrame
# Extract the 'Ingredients' column
just_ingredients = df[['IngredientsRemovedAdj']]

# Save to a new CSV file
just_ingredients.to_csv('just_ingredients.csv', index=False)

print("Ingredients column has been saved to just_ingredients.csv")


Ingredients column has been saved to just_ingredients.csv


In [39]:
import pandas as pd

# Load the dataset
df = pd.read_csv('just_ingredients.csv')

# Extract all ingredients from all rows into a single list
all_ingredients = []
for ingredients in df['IngredientsRemovedAdj']:
    all_ingredients.extend(ingredients.split(', '))

# Remove duplicates and sort the list
unique_ingredients = sorted(set(all_ingredients))

# Create a new DataFrame from the unique ingredients
df_unique_ingredients = pd.DataFrame(unique_ingredients, columns=['Ingredient'])

# Save the new DataFrame to a CSV file
df_unique_ingredients.to_csv('UniqueIngredients.csv', index=False)

print("Unique ingredients have been saved to UniqueIngredients.csv")


Unique ingredients have been saved to UniqueIngredients.csv
